In [1]:
import gspread
from gspread_dataframe import set_with_dataframe
from oauth2client.service_account import ServiceAccountCredentials

In [ ]:
# 複製貼上這一段，這是憑證認證的標準作業
scope = ['https://spreadsheets.google.com/feeds','https://www.googleapis.com/auth/drive']
credentials = ServiceAccountCredentials.from_json_keyfile_name('my-usstock-information-center-27603531a116.json', scope)
gc = gspread.authorize(credentials)

In [11]:
# 選擇試算表，我們創的試算表名稱叫做『我的投資情報專頁』
sh = gc.open('我的投資情報專頁')

# 再來選擇你要開始編輯的工作表，新創的試算表一定都會有一張原始工作表叫做『工作表1』
ws = sh.worksheet('工作表1')

# 當然也可以透過 Python 新增工作表，列數欄數依據你的需求進行設定
sh.add_worksheet(title = '新創一頁看看', rows = '5', cols = '3')

# 查看這個試算表到底有哪些工作表
sh.worksheets()

# 選擇一個工作表，然後刪除它
ws = sh.worksheet('新創一頁看看')
sh.del_worksheet(ws)

{'spreadsheetId': '11LikiO-0FBAWepZ8cnxbtoD6vF8DyjS4gANrBHMPjNI',
 'replies': [{}]}

上傳方法 1 — 指定試算表中一個格子 (cell)，上傳一筆內容

In [12]:
# 選擇要上傳的 cell，設定上傳內容
update_cell = 'A1'
update_content = '500'
ws = sh.worksheet('工作表1')
# 上傳
ws.update_acell(update_cell, update_content)

{'spreadsheetId': '11LikiO-0FBAWepZ8cnxbtoD6vF8DyjS4gANrBHMPjNI',
 'updatedRange': "'工作表1'!A1",
 'updatedRows': 1,
 'updatedColumns': 1,
 'updatedCells': 1}

上傳方法 2 — 指定一串格子，把一個 list 的資料逐一上傳

In [13]:
# 選擇要上傳的一串 cell list，設定一個 list 的上傳內容，兩者長度要一致喔！
update_cell_list = ws.range('A2:C2')
update_content_list = ['上傳資料A', '上傳資料B', '上傳資料C']

# 逐筆上傳
for i in range(len(update_content_list)):
    update_cell_list[i].value = update_content_list[i]
ws.update_cells(update_cell_list)

{'spreadsheetId': '11LikiO-0FBAWepZ8cnxbtoD6vF8DyjS4gANrBHMPjNI',
 'updatedRange': "'工作表1'!A2:C2",
 'updatedRows': 1,
 'updatedColumns': 3,
 'updatedCells': 3}

上傳方法 3 — 把整張 Dataframe 上傳到你指定的位置

In [14]:
# 隨機生成一個 dataframe 做示範
import numpy as np
import pandas as pd
update_dataframe = pd.DataFrame(np.random.randint(0,100, size = (3, 4)), index = ['商品A','商品B','商品C'], columns = ['項目1','項目2','項目3','項目4'])

# 上傳整張資料表，透過設定 row 以及 col 參數，決定這張 dataframe 的左上角第一格要指定在哪一個位置
set_with_dataframe(ws, update_dataframe, row = 4, col = 1, include_index = True, include_column_header = True)

##正式開始##
=========================
--------


In [52]:
import pandas as pd
import urllib
import urllib.request
import requests as re
from bs4 import BeautifulSoup
import yfinance as yf
from pytrends.request import TrendReq
import datetime as dt
import time
import gspread
from gspread_dataframe import set_with_dataframe
from oauth2client.service_account import ServiceAccountCredentials

先從 Slickcharts 取得股票代碼

In [53]:
import requests
import pandas as pd
# 貼上連結
url = 'https://www.slickcharts.com/sp500'
headers = {"User-Agent" : 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/83.0.4103.116 Safari/537.36'}

request = requests.get(url, headers = headers)

data = pd.read_html(request.text)[0]

# 欄位『Symbol』就是股票代碼
stk_list = data.Symbol

# 用 replace 將符號進行替換
stk_list = data.Symbol.apply(lambda x: x.replace('.', '-'))[:510]
# stk_list 存放了這 30 檔股票的股票代碼
stk_list

0       AAPL
1       MSFT
2       AMZN
3       TSLA
4      GOOGL
       ...  
499       RL
500      VNO
501     DISH
502      NWS
503     EMBC
Name: Symbol, Length: 504, dtype: object

從 Yahoo Finance 取得股票基本資料與價量資料，然後上傳！

In [54]:
# 公司基本資料與價量資料
stock = yf.Ticker('AAPL')
stk_basic_data = stock.info
info_columns = list(stk_basic_data.keys())
# stk_info_df 將存放所有股票的基本資料
stk_info_df = pd.DataFrame(index = stk_list.sort_values(), columns = info_columns)
# stk_price_df 將存放所有股票的價量資料
stk_price_df = pd.DataFrame(columns = ['label','Open','High','Low','Close','Adj Close','Volume'])

for i in stk_list:
    print('processing: ' + i)
    stock = yf.Ticker(i)
    info_dict = stock.info
    columns_included = list(info_dict.keys())
    stk_info_df.loc[i,columns_included] = list(info_dict.values())
    temp = yf.download(i, start = dt.datetime.today() + dt.timedelta(-270))
    temp['label'] = i
    # 示範頁僅呈現最後 90 交易日的資料
    temp = temp.iloc[-90:,:]
    stk_price_df = stk_price_df.append(temp)
    time.sleep(1)

processing: AAPL


C:\Users\SPY\anaconda3\envs\Learning_PY\lib\site-packages\pandas\core\internals\blocks.py:940: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  arr_value = np.asarray(value)


[*********************100%***********************]  1 of 1 completed


C:\Users\SPY\AppData\Local\Temp\ipykernel_17936\1894036511.py:20: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  stk_price_df = stk_price_df.append(temp)


processing: MSFT


...

C:\Users\SPY\AppData\Local\Temp\ipykernel_17936\1894036511.py:20: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  stk_price_df = stk_price_df.append(temp)


processing: EMBC


C:\Users\SPY\anaconda3\envs\Learning_PY\lib\site-packages\numpy\core\fromnumeric.py:3199: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  return asarray(a).ndim


[*********************100%***********************]  1 of 1 completed


C:\Users\SPY\AppData\Local\Temp\ipykernel_17936\1894036511.py:20: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  stk_price_df = stk_price_df.append(temp)


In [55]:
# 示範頁呈現的股票基本資料包含：股票名稱 /  種類 / 最後股息值 / 殖利率 / 本益比 / 近四季本益比 / $盤後漲跌幅 / 最後收盤價 / $盤後股價 / 52週最高及最低點 / 52週平均 / 52週漲跌幅  / 200天平均 / beta
try:
    stk_info_df = stk_info_df[['longName','sector','lastDividendValue','dividendYield','pegRatio','trailingEps','regularMarketPrice','fiftyTwoWeekHigh','fiftyTwoWeekLow','fiftyDayAverage','52WeekChange','twoHundredDayAverage','beta']]
except:
     stk_info_df = stk_info_df[['longName','lastDividendValue','dividendYield','pegRatio','trailingEps','regularMarketPrice','fiftyTwoWeekHigh','fiftyTwoWeekLow','fiftyDayAverage','52WeekChange','twoHundredDayAverage','beta']]

# stk_info_df[['longName','regularMarketChangePercent','preMarketChangePercent','regularMarketPrice','preMarketPrice','regularMarketDayRange','fiftyTwoWeekRange']]

stk_price_df['Date'] = list(map(lambda x: dt.datetime.strftime(x, '%Y-%m-%d'), list(stk_price_df.index)))
stk_price_df = stk_price_df[['label','Date','Open','High','Low','Close','Adj Close','Volume']]

In [56]:
stk_info_df

,longName,sector,lastDividendValue,dividendYield,pegRatio,trailingEps,regularMarketPrice,fiftyTwoWeekHigh,fiftyTwoWeekLow,fiftyDayAverage,52WeekChange,twoHundredDayAverage,beta
Symbol,,,,,,,,,,,,,
A,"Agilent Technologies, Inc.",Healthcare,None,0.0063,1.75,2.597,137.62,179.57,112.52,124.5326,None,135.1421,1.044699
AAL,American Airlines Group Inc.,Industrials,None,None,0.14,-14.725,14.17,22.35,11.93,14.0892,None,16.79535,1.548318
AAP,"Advance Auto Parts, Inc.",Consumer Cyclical,None,0.031,1.25,7.14,207.02,244.55,164,186.9592,None,210.98485,1.214559
AAPL,Apple Inc.,Technology,0.23,0.0054,3.01,6.05,171.52,182.94,129.04,151.3188,0.145682,160.461,1.230174
ABBV,AbbVie Inc.,Healthcare,None,0.0393,63.35,2.913,141.85,175.91,105.56,146.3414,None,143.2128,0.748829
...,...,...,...,...,...,...,...,...,...,...,...,...,...
YUM,"Yum! Brands, Inc.",Consumer Cyclical,None,0.0186,2.7,3.739,116.65,139.85,108.37,116.8968,None,121.5799,1.018305
ZBH,"Zimmer Biomet Holdings, Inc.",Healthcare,None,0.0087,2.36,-0.671,113.23,149.28156,100.39,108.9552,None,118.84187,1.171562
ZBRA,Zebra Technologies Corporation,Technology,None,None,1.76,9.35,328.41,615,283.72,315.9432,None,428.71234,1.606641


In [57]:
stk_price_df

,label,Date,Open,High,Low,Close,Adj Close,Volume
2022-04-12,AAPL,2022-04-12,168.020004,169.869995,166.639999,167.660004,167.181808,79265200
2022-04-13,AAPL,2022-04-13,167.389999,171.039993,166.770004,170.399994,169.913971,70618900
2022-04-14,AAPL,2022-04-14,170.619995,171.270004,165.039993,165.289993,164.818558,75329400
2022-04-18,AAPL,2022-04-18,163.919998,166.600006,163.570007,165.070007,164.599197,69023900
2022-04-19,AAPL,2022-04-19,165.020004,167.820007,163.910004,167.399994,166.922546,67723800
...,...,...,...,...,...,...,...,...
2022-08-15,EMBC,2022-08-15,30.280001,34.090000,30.000000,33.419998,33.419998,853100
2022-08-16,EMBC,2022-08-16,33.200001,33.709999,31.405001,32.330002,32.330002,415300
2022-08-17,EMBC,2022-08-17,32.160000,33.080002,31.150000,32.680000,32.680000,255700
2022-08-18,EMBC,2022-08-18,32.919998,33.529999,31.500000,33.389999,33.389999,479400


In [58]:
# 上傳 Google 試算表
# 基本認證流程，請見 Part 1 內容
scope = ['https://spreadsheets.google.com/feeds','https://www.googleapis.com/auth/drive']
credentials = ServiceAccountCredentials.from_json_keyfile_name('my-usstock-information-center-27603531a116.json', scope)
gc = gspread.authorize(credentials)
sh = gc.open('我的投資情報專頁')

# 把 stk_info_df 所有股票基本資料，上傳到倉庫 1 - 股票基本資料
sh.add_worksheet(title = '股票基本資料',rows = '1', cols = '1')
ws = sh.worksheet('股票基本資料')
set_with_dataframe(ws, stk_info_df, row = 1, col = 1, include_index = True, include_column_header = True)

# 把 stk_price_df 所有股票價量資料，上傳到倉庫 2 - 股票價量資料
sh.add_worksheet(title = '股票價量資料',rows = '1', cols = '1')
ws = sh.worksheet('股票價量資料')
set_with_dataframe(ws, stk_price_df, row = 1, col = 1, include_index = False, include_column_header = True)


APIError: {'code': 400, 'message': 'Invalid requests[0].addSheet: A sheet with the name "股票基本資料" already exists. Please enter another name.', 'status': 'INVALID_ARGUMENT'}

從 Google Trend 取得股票搜尋度指數，然後上傳！

In [ ]:
# Google 搜尋量資料
pytrends = TrendReq(hl = 'en-US', tz = 360)
# trend_df 將存放所有股票的 Google搜尋指數資料
trend_df = pd.DataFrame(columns = stk_list)

for stk in stk_list:
    try:
        print('processing : ' + stk)
        kw_list = [stk]
        pytrends.build_payload(kw_list, timeframe = 'today 3-m')
        trend_df[stk] = pytrends.interest_over_time()[stk]
        time.sleep(1)
    except:
        continue

trend_df.index = list(map(lambda x: dt.datetime.strftime(x, '%Y-%m-%d'), list(trend_df.index)))

# 上傳 Google 試算表
# 基本認證流程，請見 Part 1 內容
scope = ['https://spreadsheets.google.com/feeds','https://www.googleapis.com/auth/drive']
credentials = ServiceAccountCredentials.from_json_keyfile_name('my-usstock-information-center-27603531a116.json', scope)
gc = gspread.authorize(credentials)
sh = gc.open('我的投資情報專頁')

# 把 trend_df 所有股票 Google搜尋指數資料，上傳到倉庫 3 - 股票搜尋度指數
sh.add_worksheet(title = '股票搜尋度指數',rows = '1', cols = '1')
ws = sh.worksheet('股票搜尋度指數')
set_with_dataframe(ws, trend_df, row = 1, col = 1, include_index = True, include_column_header = True)

從 Gurufocus 取得知名投資大師持股數據，然後上傳！

In [ ]:
# 投資大師持股
# 由於每一檔股票，投資大師的數量都不一，因此先創一個空的 result 容器，後續迴圈的過程中，再把資料逐筆用 append 的方式紀錄
result = []
for j in stk_list:
    try:
        # 針對 BRK-B 這檔股票，Gurufocus 網站的代碼辨認是 BRK.B
        if j == 'BRK-B':
            j = 'BRK.B'
        print('processing: ' + j)
        soup = BeautifulSoup(re.get("https://www.gurufocus.com/stock/" + j + "/guru-trades").text, "html5lib")

        # 想爬的表格序號為 5，尋找所有 tr 標籤
        for i in range(1, len(soup.find_all('table')[5].find_all('tr'))):
            # 每一筆資料都用 dictionary 紀錄
            data = {'stk': None,
                    'investor': None,
                    'port_date': None,
                    'current_share': None,
                    'per_outstand': None,
                    'per_total_asset': None,
                    'comment': None}

            data['stk'] = j
            data['investor'] = soup.find_all('table')[5].find_all('tr')[i].find_all('td')[0].get_text().replace('\n','')
            data['port_date'] = soup.find_all('table')[5].find_all('tr')[i].find_all('td')[1].get_text()
            data['current_share'] = soup.find_all('table')[5].find_all('tr')[i].find_all('td')[2].get_text()
            data['per_outstand'] = soup.find_all('table')[5].find_all('tr')[i].find_all('td')[3].get_text()
            data['per_total_asset'] = soup.find_all('table')[5].find_all('tr')[i].find_all('td')[4].get_text()
            data['comment'] = soup.find_all('table')[5].find_all('tr')[i].find_all('td')[5].get_text()
            # 填滿這筆紀錄的所有內容後，append 到 result 容器中
            result.append(data)
    except:
        continue

# 使用 pandas 的 .from_dict 將 dictionary 轉換為 DataFrame
big_trader_df = pd.DataFrame.from_dict(result)
big_trader_df = big_trader_df[list(big_trader_df.columns)[-1:] + list(big_trader_df.columns)[:-1]]




In [ ]:
big_trader_df

In [ ]:
# 上傳 Google 試算表
# 基本認證流程，請見 Part 1 內容
scope = ['https://spreadsheets.google.com/feeds','https://www.googleapis.com/auth/drive']
credentials = ServiceAccountCredentials.from_json_keyfile_name('my-usstock-information-center-27603531a116.json', scope)
gc = gspread.authorize(credentials)
sh = gc.open('我的投資情報專頁')

# 把 big_trader_df 所有投資大師持股狀況資料，上傳到倉庫 4 - 知名投資大師的持股資料
sh.add_worksheet(title = '知名投資大師的持股資料',rows = '1', cols = '1')
ws = sh.worksheet('知名投資大師的持股資料')
set_with_dataframe(ws, big_trader_df, row = 1, col = 1, include_index = False, include_column_header = True)

從 Investing.com 取得重大經濟事件簿，然後上傳！

In [ ]:
# 重大經濟事件
# 類似於上一段的爬蟲方式
url = 'https://hk.investing.com/economic-calendar/'
r = urllib.request.Request(url)
r.add_header('User-Agent', 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/71.0.3578.98 Safari/537.36')
response = urllib.request.urlopen(r)
soup = BeautifulSoup(response.read(), 'html.parser')

# 從 html 編碼中找出目標表格！
table = soup.find('table', {'id': 'economicCalendarData'})
content = table.find('tbody').findAll('tr', {'class': 'js-event-item'})

# 如同爬取投資大師持股資料的作法將資料紀錄在 dictionary，然後 append 在 result
result = []
for i in content:
    news = {'time': None,
            'country': None,
            'impact': None,
            'event': None,
            'actual': None,
            'forecast': None,
            'previous': None}

    news['time'] = i.attrs['data-event-datetime']
    news['country'] = i.find('td', {'class': 'flagCur'}).find('span').get('title')
    news['impact'] = i.find('td', {'class': 'sentiment'}).get('title')
    news['event'] = i.find('td', {'class': 'event'}).find('a').text.strip()
    news['actual'] = i.find('td', {'class': 'bold'}).text.strip()
    news['forecast'] = i.find('td', {'class': 'fore'}).text.strip()
    news['previous'] = i.find('td', {'class': 'prev'}).text.strip()
    result.append(news)

# 同樣地使用 .from_dict 將 result 轉換成 DataFrame
event_df = pd.DataFrame.from_dict(result)
event_df = event_df[list(event_df.columns)[-1:] + list(event_df.columns)[:-1]]
event_df

In [ ]:
# 上傳 Google 試算表
# 基本認證流程，請見 Part 1 內容
scope = ['https://spreadsheets.google.com/feeds','https://www.googleapis.com/auth/drive']
credentials = ServiceAccountCredentials.from_json_keyfile_name('my-usstock-information-center-27603531a116.json', scope)
gc = gspread.authorize(credentials)
sh = gc.open('我的投資情報專頁')

# 把 event_df 今日重大經濟事件簿，上傳到倉庫 5 - 經濟事件簿資料
sh.add_worksheet(title = '經濟事件簿資料',rows = '1', cols = '1')
ws = sh.worksheet('經濟事件簿資料')
set_with_dataframe(ws, event_df, row = 1, col = 2, include_index = False, include_column_header = True)